In [28]:
import numpy as np
import nltk
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout

In [2]:
train = pd.read_csv("train.csv")

In [3]:
data = train[['text','target']].copy()

In [4]:
def removeURL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

data["text"]=data["text"].apply(lambda x: removeURL(x))

In [5]:
def removeEMO(text):
    pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return pattern.sub(r'', text)
data["text"]=data["text"].apply(lambda x: removeEMO(x))

In [6]:
def removePUN(text):
    return text.translate(str.maketrans('', '', string.punctuation))
data["text"]=data["text"].apply(lambda x: removePUN(x))

In [15]:
s = 7000

In [16]:
onehot = [one_hot(words,s) for words in data['text']]

In [23]:
sent_length =40
X = pad_sequences(onehot,padding='pre',maxlen=sent_length)
embedding_vector_features=50

Y = data['target']

In [24]:
model=Sequential()
model.add(Embedding(s,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=1)

In [27]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=30,batch_size=32)

Epoch 1/30
179/179 [==============================] - 10s 59ms/step - loss: 0.5717 - accuracy: 0.6964 - val_loss: 0.4730 - val_accuracy: 0.7810
Epoch 2/30
179/179 [==============================] - 10s 53ms/step - loss: 0.3773 - accuracy: 0.8406 - val_loss: 0.5292 - val_accuracy: 0.7684
Epoch 3/30
179/179 [==============================] - 10s 53ms/step - loss: 0.2824 - accuracy: 0.8882 - val_loss: 0.5255 - val_accuracy: 0.7589
Epoch 4/30
179/179 [==============================] - 9s 52ms/step - loss: 0.2083 - accuracy: 0.9185 - val_loss: 0.7435 - val_accuracy: 0.7437
Epoch 5/30
179/179 [==============================] - 9s 53ms/step - loss: 0.1604 - accuracy: 0.9431 - val_loss: 0.6825 - val_accuracy: 0.7458
Epoch 6/30
179/179 [==============================] - 9s 52ms/step - loss: 0.1204 - accuracy: 0.9590 - val_loss: 0.8954 - val_accuracy: 0.7421
Epoch 7/30
179/179 [==============================] - 10s 53ms/step - loss: 0.0964 - accuracy: 0.9669 - val_loss: 0.9432 - val_accuracy: 0.

In [39]:
Y_pred = model.predict_classes(X_test)

In [40]:
accuracy_score(Y_test,Y_pred)

0.7132352941176471